# Final Analysis


Aggregate results.

Plots: accuracy vs shots, accuracy vs depth, runtime vs qubits.

Compare to classical baseline.

Write short interpretation.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
sns.set(style="whitegrid", context="talk", palette="muted")
plt.rcParams["figure.figsize"] = (10, 6)

c_lr_path = "../results/quantum_log_reg_sweep_results.csv"
c_svm_path = "../results/quantum_log_reg_sweep_results.csv"
q_lr_path = "../results/quantum_log_reg_sweep_results.csv"
q_svm_path = "../results/quantum_svm_sweep_results.csv"

c_lr_df = pd.read_csv(c_lr_path)
c_svm_df = pd.read_csv(c_svm_path)
q_lr_df = pd.read_csv(q_lr_path)
q_svm_df = pd.read_csv(q_svm_path)


display(c_lr_df.head())
display(c_svm_df.head()) 
display(q_lr_df.head())
display(q_svm_df.head()) 

### Aggregate Quantum Results

In [ ]:
q_lr_agg = q_lr_df.groupby(["n_qubits", "depth", "shots", "noise"]).agg({
    "accuracy":["mean", "std"], 
    "runtime_sec": "mean"}).reset_index()

q_svm_agg = q_svm_df.groupby(["n_qubits", "depth", "shots", "noise"]).agg({
    "accuracy":["mean", "std"], 
    "runtime_sec": "mean"}).reset_index()

q_lr_agg.columns = ["n_qubits", "depth", "shots", "noise", "acc_mean", "acc_std", "runtime_sec"]
q_swm_agg.columns = ["n_qubits", "depth", "shots", "noise", "acc_mean", "acc_std", "runtime_sec"]

display(q_lr_agg)
display(q_swm_agg)

### Classical v Quantum Comparison

In [ ]:
c_lr_acc = c_lr_df["accuracy"].mean()
c_svm_acc = c_svm_df["accuracy"].mean()

c_lr_std = c_lr_df["accuracy"].std()
c_svm_std = c_svm_df["accuracy"].std()

print(f"Classical Log reg accuracy (avg): {c_lr_acc:.3f} ± {c_lr_std:.3f}")
print(f"Classical SVM accuracy (avg): {c_svm_acc:.3f} ± {c_svm_std:.3f}")

q_lr_agg["improvement_over_classical"] = q_lr_agg["acc_mean"] - c_lr_acc
q_svm_agg["improvement_over_classical"] = q_svm_agg["acc_mean"] - c_svm_acc


### Visualizations

#### Logistic Regression

**Accuracy vs Circuit Depth**

In [ ]:

plt.figure()
sns.barplot(   
    data=q_lr_agg, 
    x="depth", y="acc_mean", hue="n_qubits",
    errorbar="sd", 
    capsize=0.2  
)   

plt.title("Quantum enhanced Log Reg: Accuracy vs Circuit Depth")  
plt.ylabel("Mean Accuracy")
plt.xlabel("Circuit Depth")  
plt.legend(title="Qubits")  
plt.tight_layout()
plt.show(); 

**Accuracy vs Shots**

In [ ]:
plt.figure()  
sns.lineplot(
    data=q_lr_agg, 
    x="shots", y="acc_mean", hue="n_qubits", style="depth", markers=True
)   


plt.title("Quantum enhanced Log Reg: Accuracy vs. Num of Shots")  
plt.ylabel("Mean Accuracy") 
plt.xlabel("Shots")         
plt.xscale("log")
plt.tight_layout()  
plt.show()   

**Runtime vs Qubits**

In [ ]:
plt.figure()
sns.barplot(
    data=q_lr_agg, 
    x="n_qubits", y="runtime_sec", hue="depth",
    capsize=0.2 
)   


plt.title("Quantum enhanced Log Reg: Runtime vs. Number of Qubits")
plt.ylabel("Runtime (seconds)")
plt.xlabel("Qubits")  
plt.tight_layout()
plt.show();   

**Improvement over classical**

In [ ]:
plt.figure()
sns.heatmap(
    q_lr_agg.pivot_table(index="n_qubits", columns="depth", values="improvement_over_classical"),
    annot=True, cmap="coolwarm", center= 0  
)  

plt.title("Quantum Log Reg Improvement over Classical SVM (Accuracy)")
plt.ylabel("Qubits")
plt.xlabel("Depth")
plt.tight_layout()  
plt.show(); 

#### SVM

**Accuracy vs Circuit Depth**

In [ ]:
plt.figure()
sns.barplot(   
    data=q_svm_agg, 
    x="depth", y="acc_mean", hue="n_qubits",
    errorbar="sd", 
    capsize=0.2    
)   

plt.title("Quantum enhanced SVM: Accuracy vs Circuit Depth")  
plt.ylabel("Mean Accuracy")
plt.xlabel("Circuit Depth")  
plt.legend(title="Qubits")     
plt.tight_layout()
plt.show(); 

**Accuracy vs Shots**

In [ ]:
plt.figure()  
sns.lineplot(
    data=q_svm_agg, 
    x="shots", y="acc_mean", hue="n_qubits", style="depth", markers=True
)  


plt.title("Quantum enhanced SVM: Accuracy vs. Num of Shots")  
plt.ylabel("Mean Accuracy") 
plt.xlabel("Shots")         
plt.xscale("log")
plt.tight_layout()  
plt.show()   

**Runtime vs Qubits**

In [ ]:
plt.figure()
sns.barplot(
    data=q_svm_agg, 
    x="n_qubits", y="runtime_sec", hue="depth",
    capsize=0.2 
)   


plt.title("Quantum enhanced SVM: Runtime vs. Number of Qubits")
plt.ylabel("Runtime (seconds)")
plt.xlabel("Qubits")  
plt.tight_layout()
plt.show();   

**Improvement over classical**

In [ ]:
plt.figure()
sns.heatmap(
    q_svm_agg.pivot_table(index="n_qubits", columns="depth", values="improvement_over_classical"),
    annot=True, cmap="coolwarm", center= 0  
)  

plt.title("Quantum SVM Improvement over Classical SVM (Accuracy)")
plt.ylabel("Qubits")
plt.xlabel("Depth")
plt.tight_layout() 
plt.show(); 

### Save Summary

In [ ]:
summary_path = "../results/metrics/quantum_summary.csv"
q_lr_agg.to_csv("../results/metrics/q_lr_summary.csv", index=False)
q_svm_agg.to_csv("../results/metrics/q_svm_summary.csv", index=False)